In [1]:
import pandas as pd
import itertools
from rdkit import Chem
from rdkit.Chem import AllChem
import copy
import json
import os

# bunch of code

In [31]:
class MapRules:
    """Map intended reactants to database reactants described by the same reaction operators."""

    def __init__(self, atom_mapping_path=None, rules_path=None, molfiles_path=None, seed_dict=None,
                 cofactor_list_path=None, cofactor_pair_path=None, cofactors=True, start_name=None, start_smiles=None):
        self.atom_mapping_path = atom_mapping_path
        self.rules = pd.read_csv(rules_path, sep='\t', index_col=0)
        self.molfiles_path = molfiles_path
        if self.atom_mapping_path:
            self.reaction_df = self._create_metacyc_info(self.atom_mapping_path)
        if cofactors:
            self.cofactor_name_dict, self.cofactor_list_dict, self.cofactor_pair_dict = get_cofactors(cofactor_list_path, cofactor_pair_path)
        else:
            self.cofactor_name_dict = {}
            self.cofactor_list_dict = {}
            self.cofactor_pair_dict = {}
        self.seed_dict = seed_dict
        if start_name:
            self.start_name = start_name
        if start_smiles:
            self.start_smiles = start_smiles

    def _create_metacyc_info(self, atom_mapping_path):
        """Read metacyc atom mapping information and return as a dataframe, also takes in reaction.dat information"""

        # read reactions.dat, group info for each reaction
        with open(atom_mapping_path, 'r') as file:
            reactions = file.read().split('//')

        # read reactions.dat info for each reaction
        info_df = pd.DataFrame()

        for entry in reactions:

            # storing info for this reaction
            info_dict = {}

            # for each line
            lines = entry.split('\n')
            for line in lines:
                try:
                    line = line.split(' - ', 1)
                    if (line[0] not in info_dict) & (line[1] != 'PROTON') & (line[1] != '|Acceptor|') \
                            & (line[1] != '|Donor-H2|') & (line[1] != '|ETR-Quinones|') & (line[1] != '|ETR-Quinols|'):
                        # if left or right in atom-mappings
                        if line[0] == 'LEFT' or line[0] == 'RIGHT':
                            try:
                                if line[1] + ' ' in info_dict['ATOM-MAPPINGS']:
                                    info_dict[line[0]] = line[1]
                            except KeyError:
                                pass
                        else:
                            info_dict[line[0]] = line[1]
                except:
                    continue

            # process reaction information
            try:
                unique_id = info_dict['UNIQUE-ID']
            except KeyError:
                continue

            try:
                reaction_direction = info_dict['REACTION-DIRECTION'].replace('PHYSIOL-', '').replace('IRREVERSIBLE-',
                                                                                                     '')
                left = info_dict['LEFT'].lstrip('|').rstrip('|') + ' '
                right = info_dict['RIGHT'].lstrip('|').rstrip('|') + ' '
            except KeyError:
                reaction_direction = ''
                left = ''
                right = ''

            try:
                ec = info_dict['EC-NUMBER'].lstrip('EC-')
            except KeyError:
                ec = ''

            # add information to info_df
            info_add = pd.DataFrame([left, right, reaction_direction, ec], index=['Left', 'Right', 'Direction', 'EC'],
                                    columns=[unique_id])
            info_df = pd.concat([info_df, info_add], axis=1, sort=True)

        # read atom mapping, group info for each reaction
        with open(atom_mapping_path, 'r') as file:
            metacyc = file.read().split('//')

        # read atom mapping info for each reaction
        reaction_df = pd.DataFrame()
        for entry in metacyc:

            # storing info for this reaction
            reaction_dict = {}

            # for each line
            lines = entry.split('\n')
            for line in lines:
                try:
                    line = line.split(' - ', 1)
                    reaction_dict[line[0]] = line[1]
                except:
                    continue

            # solve for MetaCyc 19.0 ((
            try:
                am = reaction_dict['ATOM-MAPPINGS'].lstrip('(').split('(')
                am = '('.join(am[1:])

                if not am:  # skip if no am
                    continue

                # parse am, reactants, products
                temp = am.split(')) ((')
                product_temp = '(' + temp[1].rstrip(')') + ')'

                temp = temp[0].split(') (')
                am_temp = temp[0]
                reactant_temp = '(' + ') ('.join(temp[1:]).replace('(', '', 1).replace('(', '', 1) + ')'

                # duplicate mols
                if '((' in reactant_temp:  # reactant side
                    name_edit = reactant_temp
                    name_edit = name_edit.split('((')

                    for i in range(1, len(name_edit)):  # edit each duplicate mol
                        name_edit[i] = name_edit[i].replace(' ', ':', 1).replace(')', '', 1)

                    reactant_temp = '('.join(name_edit)

                if '((' in product_temp:  # product side
                    name_edit = product_temp
                    name_edit = name_edit.split('((')

                    for i in range(1, len(name_edit)):  # edit each duplicate mol
                        name_edit[i] = name_edit[i].replace(' ', ':', 1).replace(')', '', 1)

                    product_temp = '('.join(name_edit)

                reactant_temp = ''.join(re.findall(r'[A-Za-z0-9_:()\- ]', reactant_temp))
                product_temp = ''.join(re.findall(r'[A-Za-z0-9_:()\- ]', product_temp))

            except KeyError:
                continue

            # determine if UNIQUE-ID in atom-mapping is in reactions.dat
            try:
                info_dict = info_df[reaction_dict['UNIQUE-ID']]
            except KeyError:
                direction = '+-'
                ec = ''
                reaction_add = pd.DataFrame(
                    [reactant_temp, product_temp, am_temp, direction, ec],
                    index=['Reactants', 'Products', 'AM', 'Direction', 'EC'], columns=[reaction_dict['UNIQUE-ID']])
                reaction_df = pd.concat([reaction_df, reaction_add], axis=1, sort=True)
                continue

            # process direction info
            direction = info_dict.Direction
            ec = info_dict.EC

            if direction != '':  # if direction info is provided
                if direction == 'REVERSIBLE':  # if reversible
                    direction = '+-'
                elif direction == 'LEFT-TO-RIGHT':  # if left to right
                    if (info_dict.Left in reactant_temp) & (info_dict.Right in product_temp):
                        direction = '+'
                    elif (info_dict.Right in reactant_temp) & (info_dict.Left in product_temp):
                        direction = '-'
                elif direction == 'RIGHT-TO-LEFT':  # if right to left
                    if (info_dict.Left in reactant_temp) & (info_dict.Right in product_temp):
                        direction = '-'
                    elif (info_dict.Right in reactant_temp) & (info_dict.Left in product_temp):
                        direction = '+'
                else:
                    direction = '+-'
            else:  # if direction info not provided, default reversible
                direction = '+-'

            reaction_add = pd.DataFrame(
                [reactant_temp, product_temp, am_temp, direction, ec],
                index=['Reactants', 'Products', 'AM', 'Direction', 'EC'], columns=[reaction_dict['UNIQUE-ID']])
            reaction_df = pd.concat([reaction_df, reaction_add], axis=1, sort=True)

        print('Finished reading MetaCyc information.')

        return reaction_df

    def _read_cofactor_info(self, cofactor_list_path, cofactor_pair_path):
        """Read in cofactor information, store so that reactants can be accurately mapped later
        :return: cofactor_name_dict (all cofactors)
        :rtype: {name in rxn rule: [names of molfile]}
        :return: cofactor_list_dict (cofactor list)
        :rtype: {cofactor: [names of molfile]}
        :return: cofactor_pair_dict (cofactor pair)
        :rtype: {cofactor1___cofactor2: [name of molfile1___name of molfile2]}
        """

        cofactor_name_dict = {}
        cofactor_list_dict = {}
        cofactor_pair_dict = {}

        cofactor_list_df = pd.read_csv(cofactor_list_path)
        cofactor_pair_df = pd.read_csv(cofactor_pair_path)

        # for each list
        for i, current_df in cofactor_list_df.iterrows():
            try:
                cofactor_name_dict[current_df['replacement']].append(current_df['molfile'])
            except KeyError:
                cofactor_name_dict[current_df['replacement']] = [current_df['molfile']]
            try:
                cofactor_list_dict[current_df['replacement']].append(current_df['molfile'])
            except KeyError:
                cofactor_list_dict[current_df['replacement']] = [current_df['molfile']]

        # for each pair
        for i, current_df in cofactor_pair_df.iterrows():
            try:
                cofactor_name_dict[current_df['reactant_replacement']].append(current_df['reactant_molfile'])
            except KeyError:
                cofactor_name_dict[current_df['reactant_replacement']] = [current_df['reactant_molfile']]
            try:
                cofactor_pair_dict[
                    current_df['reactant_replacement'] + '___' + current_df['product_replacement']].append(
                    current_df['reactant_molfile'] + '___' + current_df['product_molfile'])
            except KeyError:
                cofactor_pair_dict[
                    current_df['reactant_replacement'] + '___' + current_df['product_replacement']] =\
                    [current_df['reactant_molfile'] + '___' + current_df['product_molfile']]

        # unique molfile name
        for k, v in cofactor_name_dict.items():
            cofactor_name_dict[k] = sorted(list(set(v)))
        for k, v in cofactor_pair_dict.items():
            cofactor_pair_dict[k] = sorted(list(set(v)))

        return cofactor_name_dict, cofactor_list_dict, cofactor_pair_dict

    def map_pickaxe_rules(self, lhs_dict, rhs_dict, rule_current, return_reaction_center=False):

        rxn_df = self.rules.loc[rule_current.split(';')[0]]
        rule = rxn_df['SMARTS']
        reactants = rxn_df['Reactants']
        products = rxn_df['Products']

        # remove cofactor, sanitize mols
        lhs_list, rhs_list = self._process_substrates(lhs_dict, rhs_dict, rule_current)
        print(lhs_list, rhs_list)

        # match index with pickaxe
        match_index = self._map_rules(rule, lhs_list, rhs_list, reactants, products, return_reaction_center)

        return match_index

    def _map_rules(self, rule, lhs, rhs, reactants, products, return_reaction_center):
        """Operator mapping"""

        rxn = Chem.rdChemReactions.ReactionFromSmarts(rule)
        reactants = reactants.split(';')
        cofactor_index_reactants = [i for i, r in enumerate(reactants) if r != 'Any']

        products = products.split(';')
        cofactor_index_products = [i for i, p in enumerate(products) if p != 'Any']

        # if number of reactants does not match reactant template
        if len(lhs) > reactants.count('Any'):
            repetitive_mols = set(lhs).intersection(set(rhs))

            while repetitive_mols:
                lhs.remove(sorted(repetitive_mols)[0])
                rhs.remove(sorted(repetitive_mols)[0])
                repetitive_mols = set(lhs).intersection(set(rhs))

        lhs_set = set()
        for lhs_perm in itertools.permutations(lhs):
            lhs_set.add(lhs_perm)

        for lhs_perm in lhs_set:
            lhs_temp = list(lhs_perm)

            for c in cofactor_index_reactants:
                if self.molfiles_path:
                    lhs_temp[c:c] = [Chem.MolToSmiles(Chem.MolFromMolFile(os.path.sep.join([self.molfiles_path, self.cofactor_name_dict[reactants[c]] + '.mol'])))]
                elif self.seed_dict:
                    lhs_temp[c:c] = [self.seed_dict[self.cofactor_name_dict[reactants[c]]]]

            # pruned MetaCyc
            try:
                lhs_tuple = tuple([Chem.MolFromSmiles(i) for i in lhs_temp])
                outputs = rxn.RunReactants(lhs_tuple)
            except:
                try:
                    lhs_tuple = tuple([Chem.MolFromSmiles(i, sanitize=False) for i in lhs_temp])
                    outputs = rxn.RunReactants(lhs_tuple)
                except:
                    continue

            # # pickaxe
            # lhs_tuple_list = []
            # for i in lhs_temp:
            #     try:
            #         temp_mol = Chem.MolFromSmiles(i)
            #         temp_mol = AllChem.AddHs(temp_mol)
            #         AllChem.Kekulize(temp_mol, clearAromaticFlags=True)
            #     except:
            #         temp_mol = Chem.MolFromSmiles(i, sanitize=False)
            #     lhs_tuple_list.append(temp_mol)
            # lhs_tuple = tuple(lhs_tuple_list)
            # outputs = rxn.RunReactants(lhs_tuple)

            for rxn_output in outputs:

                rhs_run = [Chem.MolToSmiles(rhs_mols) for rhs_mols in rxn_output]
                rhs_list = copy.deepcopy(rhs_run)

                for c in cofactor_index_products:
                    rhs_list.remove(rhs_run[c])

                # for all tautomer possibilities of clean rhs
                for rhs in postsanitize_smiles(rhs):
                    rhs = list(rhs)

                    for rhs_list in postsanitize_smiles(rhs_list):

                        # pruned MetaCyc
                        if sorted(list(rhs_list)) == sorted(rhs):
                            # lhs_index = [int(np.where(np.argsort(lhs) == i)[0]) for i in np.argsort(lhs_perm)]
                            # rhs_index = [int(np.where(np.argsort(rhs) == i)[0]) for i in np.argsort(rhs_list)]
                            lhs_index = [lhs.index(i) for i in lhs_perm]
                            rhs_index = [rhs.index(i) for i in rhs_list]

                            # return atom index of reaction center
                            if return_reaction_center:

                                # try to append lhs reactants
                                lhs_mols = []
                                for l in lhs_perm:
                                    lhs_mols.append(Chem.MolFromSmiles(l))
                                    if not lhs_mols[-1]:
                                        lhs_mols[-1] = Chem.MolFromSmiles(l, sanitize=False)

                                smarts_list, _ = get_smarts(rule)
                                smarts_list = [s for i, s in enumerate(smarts_list)
                                               if i not in cofactor_index_reactants]

                                # possible reaction center
                                temp_lhs_match = [Chem.MolFromSmiles(l, sanitize=False).GetSubstructMatches(
                                    Chem.MolFromSmarts(smarts_list[i])) for i, l in enumerate(lhs_perm)]
                                reaction_center_set = [set(itertools.chain(*l)) for l in temp_lhs_match]
                                lhs_all_matches = itertools.product(*temp_lhs_match)

                                # for all possible reaction centers
                                for lhs_match in lhs_all_matches:

                                    # iterate over all reactants
                                    for l_idx, match in enumerate(lhs_match):
                                        for protect in reaction_center_set[l_idx] - set(match):
                                            lhs_mols[l_idx].GetAtomWithIdx(protect).SetProp('_protected', '1')

                                    # add cofactors
                                    lhs_temp_mol = list(lhs_mols)
                                    for c in cofactor_index_reactants:
                                        if self.molfiles_path:
                                            lhs_temp_mol[c:c] = [Chem.MolFromMolFile(os.path.sep.join(
                                                [self.molfiles_path, self.cofactor_name_dict[reactants[c]] + '.mol']))]
                                        elif self.seed_dict:
                                            lhs_temp_mol[c:c] = [Chem.MolFromSmiles(
                                                self.seed_dict[self.cofactor_name_dict[reactants[c]]])]

                                    # for all possible reaction outcomes
                                    for rhs_rxn in rxn.RunReactants(tuple(lhs_temp_mol)):

                                        for rhs_smiles in postsanitize_smiles([Chem.MolToSmiles(r) for r in rhs_rxn]):

                                            # found match
                                            if tuple(r for i, r in enumerate(rhs_smiles)
                                                     if i not in cofactor_index_products) == rhs_list:
                                                return lhs_index, rhs_index, list(lhs_match)

                                    # else remove protection
                                    for l_idx, match in enumerate(lhs_match):
                                        for deprotect in reaction_center_set[l_idx] - set(match):
                                            lhs_mols[l_idx].GetAtomWithIdx(deprotect).ClearProp('_protected')

                            else:
                                return lhs_index, rhs_index

                        # # pickaxe
                        # if sorted(list(rhs_list)) == sorted(rhs):
                        #     lhs_index = [int(np.where(np.argsort(lhs) == i)[0]) for i in np.argsort(lhs_perm)]
                        #     rhs_index = [int(np.where(np.argsort(rhs) == i)[0]) for i in np.argsort(rhs_list)]
                        #     return lhs_index, rhs_index

        return None, None

    def _post_process(self, enzyme_list):
        """Post processing"""

        enzyme_list_temp = []

        non_orphan_flag = False

        for e in enzyme_list:
            if 'ENZRXN' in e:
                non_orphan_flag = True
                continue

        if non_orphan_flag:
            for e in enzyme_list:
                if 'ENZRXN' in e:
                    enzyme_list_temp.append(e)
            return enzyme_list_temp
        else:
            return enzyme_list

    def _process_substrates(self, lhs_dict_temp, rhs_dict_temp, rule_current):
        """Process substrates"""

        # check cofactor designation
        rule_reactant_names = self.rules.loc[rule_current, 'Reactants']
        rule_product_names = self.rules.loc[rule_current, 'Products']

        reactant_names, product_names = label_cofactor(sorted(lhs_dict_temp), sorted(rhs_dict_temp), self.cofactor_list_dict, self.cofactor_pair_dict)
        print('from label_cofactor', reactant_names, product_names)
        if sorted(rule_reactant_names.split(';')) != sorted(reactant_names.split(';')) \
                or sorted(rule_product_names.split(';')) != sorted(product_names.split(';')):
            raise ValueError('Cofactor designation error.')

        # create list from dict
        lhs_list = [lhs_dict_temp[k] for i, k in enumerate(sorted([k for k in lhs_dict_temp]))
                    if reactant_names.split(';')[i] == 'Any']
        rhs_list = [rhs_dict_temp[k] for i, k in enumerate(sorted([k for k in rhs_dict_temp]))
                    if product_names.split(';')[i] == 'Any']

        # sanitize
        for i, m in enumerate(lhs_list):
            try:
                temp_mol = Chem.MolFromSmiles(m)
                Chem.rdmolops.RemoveStereochemistry(temp_mol)
                lhs_list[i] = Chem.MolToSmiles(temp_mol)
            except:
                temp_mol = Chem.MolFromSmiles(m, sanitize=False)
                Chem.rdmolops.RemoveStereochemistry(temp_mol)
                lhs_list[i] = Chem.MolToSmiles(temp_mol)

        for i, m in enumerate(rhs_list):
            try:
                temp_mol = Chem.MolFromSmiles(m)
                Chem.rdmolops.RemoveStereochemistry(temp_mol)
                rhs_list[i] = Chem.MolToSmiles(temp_mol)
            except:
                temp_mol = Chem.MolFromSmiles(m, sanitize=False)
                Chem.rdmolops.RemoveStereochemistry(temp_mol)
                rhs_list[i] = Chem.MolToSmiles(temp_mol)

        return lhs_list, rhs_list
    
def get_cofactors(input_cofactor_list_path, input_cofactor_pair_path):
    """Get cofactor list & pairs"""

    # cofactor to cpd id dict
    cofactor_name_dict = {}

    # cofactor list name designation
    cofactor_list_dict = {}
    for k, v in pd.read_csv(input_cofactor_list_path, sep='\t', index_col=0).iterrows():
        cofactor_list_dict[k.upper()] = v['replacement']
        if v['replacement'] not in cofactor_name_dict:
            cofactor_name_dict[v['replacement']] = k

    # cofactor pair name designation
    cofactor_pair_dict = {}
    with open(input_cofactor_pair_path) as f:
        cofactor_pair_read_json = json.loads(f.read())
    for k, v in cofactor_pair_read_json.items():
        for pair in v:
            cofactor_pair_dict[(pair[1].upper(), pair[2].upper())] = k
            cofactor_pair_dict[(pair[2].upper(), pair[1].upper())] = '%s,%s' % (k.split(',')[1], k.split(',')[0])
            if k.split(',')[0] not in cofactor_name_dict:
                cofactor_name_dict[k.split(',')[0]] = pair[1]
                cofactor_name_dict[k.split(',')[1]] = pair[2]

    return cofactor_name_dict, cofactor_list_dict, cofactor_pair_dict

def label_cofactor(input_reactant_molfile, input_product_molfile, cofactor_list, cofactor_pair):  # label cofactors
    """Label cofactors & cofator pairs for product & reactant names"""

    # reactant & product names
    reactant_molfile = [':'.join(m.upper().split(':')[0:max(1, len(m.split(':')) - 1)]) for m in input_reactant_molfile]
    product_molfile = [':'.join(m.upper().split(':')[0:max(1, len(m.split(':')) - 1)]) for m in input_product_molfile]

    # new substrate labels
    reactant_names = ['Any'] * len(reactant_molfile)
    product_names = ['Any'] * len(product_molfile)

    # get cofactor pairs
    for i_lhs, lhs in enumerate(reactant_molfile):
        for i_rhs, rhs in enumerate(product_molfile):

            # skip if already assigned
            if product_names[i_rhs] != 'Any':
                continue

            # assign cofactor pair designation
            try:
                temp_pair = cofactor_pair[(lhs, rhs)]
                reactant_names[i_lhs] = temp_pair.split(',')[0]
                product_names[i_rhs] = temp_pair.split(',')[1]
                break
            except KeyError:
                continue

        # assign cofactor list if no cofactor pair assigned
        if reactant_names[i_lhs] == 'Any':
            try:
                reactant_names[i_lhs] = cofactor_list[lhs]
            except KeyError:
                continue

    # assign cofactor list for rhs
    for i_rhs, rhs in enumerate(product_molfile):

        # assign cofactor list if no cofactor pair assigned
        if product_names[i_rhs] == 'Any':
            try:
                product_names[i_rhs] = cofactor_list[rhs]
            except KeyError:
                continue

    return ';'.join(reactant_names), ';'.join(product_names)

def postsanitize_smiles(smiles_list):
    """Postsanitize smiles after running SMARTS.
    :returns tautomer list of list of smiles"""

    sanitized_list = []
    # tautomer_smarts = '[#6:1]1:[#6:2]:[#7H1X3:3]:[#6:4]:[#7H0X2:5]:1>>[#6:1]1:[#6:2]:[#7H0X2:3]:[#6:4]:[#7H1X3:5]:1'
    tautomer_smarts = '[#7H1X3&a:1]:[#6&a:2]:[#7H0X2&a:3]>>[#7H0X2:1]:[#6:2]:[#7H1X3:3]'

    for s in smiles_list:

        temp_mol = Chem.MolFromSmiles(s, sanitize=False)

        # # pickaxe
        # temp_mol = Chem.rdmolops.RemoveHs(temp_mol)

        aromatic_bonds = [i.GetIdx() for i in temp_mol.GetBonds() if i.GetBondType() == Chem.rdchem.BondType.AROMATIC]

        for i in temp_mol.GetBonds():
            if i.GetBondType() == Chem.rdchem.BondType.UNSPECIFIED:
                i.SetBondType(Chem.rdchem.BondType.SINGLE)

        try:
            Chem.SanitizeMol(temp_mol)
            Chem.rdmolops.RemoveStereochemistry(temp_mol)
            temp_smiles = Chem.MolToSmiles(temp_mol)

        except Exception as msg:
            if 'Can\'t kekulize mol' in str(msg):
                # unkekulized_indices = [int(i) for i in str(msg).split('Unkekulized atoms: ')[1].split('.')[0].rstrip(' \n').split(' ')]
                pyrrole_indices = [i[0] for i in temp_mol.GetSubstructMatches(Chem.MolFromSmarts('n'))]

                # indices to sanitize
                # for s_i in set(unkekulized_indices).intersection(set(pyrrole_indices)):
                for s_i in pyrrole_indices:
                    temp_mol = Chem.MolFromSmiles(s, sanitize=False)
                    if temp_mol.GetAtomWithIdx(s_i).GetNumExplicitHs() == 0:
                        temp_mol.GetAtomWithIdx(s_i).SetNumExplicitHs(1)
                    elif temp_mol.GetAtomWithIdx(s_i).GetNumExplicitHs() == 1:
                        temp_mol.GetAtomWithIdx(s_i).SetNumExplicitHs(0)
                    try:
                        Chem.SanitizeMol(temp_mol)

                        processed_pyrrole_indices = [i[0] for i in
                                                     temp_mol.GetSubstructMatches(Chem.MolFromSmarts('n'))]
                        processed_aromatic_bonds = [i.GetIdx() for i in
                                                    temp_mol.GetBonds() if i.GetBondType() == Chem.rdchem.BondType.AROMATIC]
                        if processed_pyrrole_indices != pyrrole_indices or aromatic_bonds != processed_aromatic_bonds:
                            continue

                        Chem.rdmolops.RemoveStereochemistry(temp_mol)
                        temp_smiles = Chem.MolToSmiles(temp_mol)
                        break
                    except:
                        continue
                if 'temp_smiles' not in vars():
                    Chem.rdmolops.RemoveStereochemistry(temp_mol)
                    temp_smiles = Chem.MolToSmiles(temp_mol)
                    sanitized_list.append([temp_smiles])
                    continue
            else:
                Chem.rdmolops.RemoveStereochemistry(temp_mol)
                temp_smiles = Chem.MolToSmiles(temp_mol)
                sanitized_list.append([temp_smiles])
                continue
        rxn = AllChem.ReactionFromSmarts(tautomer_smarts)

        try:
            tautomer_mols = rxn.RunReactants((Chem.MolFromSmiles(temp_smiles), ))
        except:
            try:
                tautomer_mols = rxn.RunReactants((Chem.MolFromSmiles(temp_smiles, sanitize=False),))
            except:
                continue

        tautomer_smiles = [Chem.MolToSmiles(m[0]) for m in tautomer_mols]
        sanitized_list.append(sorted(set(tautomer_smiles + [temp_smiles])))

    return list(itertools.product(*sanitized_list))

# initialize reaction mapper

In [32]:
cpd_path = 'brenda_neutralize.tsv'
rules_path = 'minimal1224_all_uniprot.tsv'
#rules_path = 'JN3604IMT_rules.tsv'
cofactor_list_path = 'cofactor_list_alldb.tsv'
cofactor_pair_path = 'cofactor_pair_alldb.json'
SEED_neutralized = 'SEED_neutralized.tsv'
db_cpd_dict = {k: v['smiles'] for k, v in pd.read_csv(cpd_path, sep='\t', index_col=0).iterrows()}
mapper = MapRules(rules_path=rules_path, cofactor_list_path=cofactor_list_path, cofactor_pair_path=cofactor_pair_path, seed_dict=db_cpd_dict)

In [33]:
### Reactants
Deoxyerythronolide_SMILES = 'CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)CC(C)C(=O)C(C)C(O)C1C'

Deoxyerythronolide_SEED_ID = 'cpd02075:0'

oxygen_smiles = 'O=O'
oxygen_SEED_ID = 'cpd00007:0'

NADH_smiles = 'NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1'
NADH_SEED_ID = 'cpd00004:0'

### Products
erythronolide_SMILES = 'CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(=O)C(C)C(O)C1C'

erythronolide_SEED_ID = 'cpd04039:0'

water_SMILES = 'O'
water_SEED_ID = 'cpd00001:0'

NAD_SEED_ID = 'cpd00003:0'
NAD_SMILES = 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)c1'

rxn_dict = {'R0001':
            [
                ### Reactants dictionary
                {Deoxyerythronolide_SEED_ID:Deoxyerythronolide_SMILES,
                 NADH_SEED_ID:NADH_smiles,
                 oxygen_SEED_ID:oxygen_smiles},

                ### Products dictionary
                {erythronolide_SEED_ID:erythronolide_SMILES,
                 NAD_SEED_ID:NAD_SMILES,
                 water_SEED_ID:water_SMILES}]}

lhs_rhs = rxn_dict['R0001']
mapper.map_pickaxe_rules(lhs_rhs[0], lhs_rhs[1], 'rule0004')

from label_cofactor NADH_CoF;O2;Any WATER;NAD_CoF;Any
['CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)CC(C)C(=O)C(C)C(O)C1C'] ['CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(=O)C(C)C(O)C1C']


RDKit ERROR: [11:13:02] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [11:13:02] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [11:13:02] Explicit valence for atom # 4 C, 5, is greater than permitted
[11:13:02] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [11:13:02] Explicit valence for atom # 4 C, 5, is greater than permitted
[11:13:02] Explicit valence for atom # 4 C, 5, is greater than permitted
[11:13:02] Explicit valence for atom # 4 C, 5, is greater than permitted
[11:13:02] Explicit valence for atom # 4 C, 5, is greater than permitted


([0], [0])

In [54]:
SEED_neutralized_df = pd.read_csv('SEED_neutralized.tsv')

# dict to store all reactions

In [55]:
rxn_dict = {'R00465': 
            [{'cpd00006:0': 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)c1', 'cpd00139:0': 'O=C(O)CO'},
             {'cpd00005:0': 'NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1', 'cpd00040:0': 'O=CC(=O)O'},
             'rule0002']}

In [56]:
lhs_rhs = rxn_dict['R00465']
lhs_rhs[0]

{'cpd00006:0': 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)c1',
 'cpd00139:0': 'O=C(O)CO'}

In [57]:
lhs_rhs[1]

{'cpd00005:0': 'NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1',
 'cpd00040:0': 'O=CC(=O)O'}

In [58]:
lhs_rhs[0]

{'cpd00006:0': 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)c1',
 'cpd00139:0': 'O=C(O)CO'}

In [4]:
### Reactants
Deoxyerythronolide_SMILES = 'CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)CC(C)C(=O)C(C)C(O)C1C'

Deoxyerythronolide_SEED_ID = 'cpd02075:0'

oxygen_smiles = 'O=O'
oxygen_SEED_ID = 'cpd00007:0'

NADH_smiles = 'NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1'
NADH_SEED_ID = 'cpd00004:0'

### Products
erythronolide_SMILES = 'CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(=O)C(C)C(O)C1C'

erythronolide_SEED_ID = 'cpd04039:0'

water_SMILES = 'O'
water_SEED_ID = 'cpd00001:0'

NAD_SEED_ID = 'cpd00003:0'
NAD_SMILES = 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)c1'

rxn_dict = {'R0001':
            [
                ### Reactants dictionary
                {Deoxyerythronolide_SEED_ID:Deoxyerythronolide_SMILES,
                 NADH_SEED_ID:NADH_smiles,
                 oxygen_SEED_ID:oxygen_smiles},

                ### Products dictionary
                {erythronolide_SEED_ID:erythronolide_SMILES,
                 NAD_SEED_ID:NAD_SMILES,
                 water_SEED_ID:water_SMILES}]}

# examples

In [34]:
def map_rxn_to_rule(lhs_rhs):
    count = 0
    for i in range(0,2000):

        count += 1
        if count<10:
            rule = 'rule000%i'%count
        elif count >=10 and count<100:
            rule = 'rule00%i'%count
        elif count >=100 and count<1000:
            rule = 'rule0%i'%count
        else:
            rule = 'rule%i'%count

        try:
            mapping_object = mapper.map_pickaxe_rules(lhs_rhs[0], lhs_rhs[1],rule)
            if mapping_object[0][0] == 0 and mapping_object[1][0] == 0:
                break
        except ValueError: # exception for specific error type
            pass

    return rule

mapped reaction

In [35]:
lhs_rhs = rxn_dict['R0001']
map_rxn_to_rule(lhs_rhs)

from label_cofactor NADH_CoF;O2;Any WATER;NAD_CoF;Any
from label_cofactor NADH_CoF;O2;Any WATER;NAD_CoF;Any
from label_cofactor NADH_CoF;O2;Any WATER;NAD_CoF;Any
from label_cofactor NADH_CoF;O2;Any WATER;NAD_CoF;Any
['CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)CC(C)C(=O)C(C)C(O)C1C'] ['CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(=O)C(C)C(O)C1C']


RDKit ERROR: [11:23:55] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [11:23:55] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [11:23:55] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [11:23:55] Explicit valence for atom # 4 C, 5, is greater than permitted
[11:23:55] Explicit valence for atom # 4 C, 5, is greater than permitted
[11:23:55] Explicit valence for atom # 4 C, 5, is greater than permitted
[11:23:55] Explicit valence for atom # 4 C, 5, is greater than permitted
[11:23:55] Explicit valence for atom # 4 C, 5, is greater than permitted


'rule0004'

In [38]:
lhs_rhs = rxn_dict['R0001']

In [18]:
mapper.map_pickaxe_rules(lhs_rhs[0], lhs_rhs[1], 'rule0004')

hello ['CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)CC(C)C(=O)C(C)C(O)C1C']


RDKit ERROR: [16:59:06] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [16:59:06] Explicit valence for atom # 4 C, 5, is greater than permitted
[16:59:06] Explicit valence for atom # 4 C, 5, is greater than permitted
[16:59:06] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [16:59:06] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [16:59:06] Explicit valence for atom # 4 C, 5, is greater than permitted
[16:59:06] Explicit valence for atom # 4 C, 5, is greater than permitted
[16:59:06] Explicit valence for atom # 4 C, 5, is greater than permitted


([0], [0])

In [7]:
mapper.map_pickaxe_rules(lhs_rhs[0], lhs_rhs[1], 'rule0002')

NameError: name 'mapper' is not defined

In [65]:
print(mapper.map_pickaxe_rules(lhs_rhs[0], lhs_rhs[1], 'rule0002'))

([0], [0])


RDKit ERROR: [15:57:30] Explicit valence for atom # 1 C, 5, is greater than permitted
RDKit ERROR: [15:57:30] Explicit valence for atom # 1 C, 5, is greater than permitted


unmapped reaction

In [66]:
print(mapper.map_pickaxe_rules(lhs_rhs[0], lhs_rhs[1], 'rule0018'))==None

(None, None)


RDKit ERROR: [16:27:23] Explicit valence for atom # 1 C, 5, is greater than permitted
RDKit ERROR: [16:27:23] Explicit valence for atom # 1 C, 5, is greater than permitted
RDKit ERROR: [16:27:23] Explicit valence for atom # 1 C, 5, is greater than permitted
RDKit ERROR: [16:27:23] Explicit valence for atom # 1 C, 5, is greater than permitted


True

ValueError if cofactor not matched

In [11]:
print(mapper.map_pickaxe_rules(lhs_rhs[0], lhs_rhs[1], 'rule0001'))

ValueError: Cofactor designation error.

### Yash

In [58]:
table6_filepath = "/Users/yashchainani96/PycharmProjects/enzyme_substrate_data/table6_reactants_and_products.json"
table7_filepath = "/Users/yashchainani96/PycharmProjects/enzyme_substrate_data/table7_reactants_and_products.json"
table8_filepath = "/Users/yashchainani96/PycharmProjects/enzyme_substrate_data/table8_reactants_and_products.json"

In [59]:
with open(table6_filepath, 'r') as j:
     table6_dict = json.loads(j.read())
with open(table7_filepath, 'r') as j:
     table7_dict = json.loads(j.read())
with open(table8_filepath,'r') as j:
     table8_dict = json.loads(j.read())

In [21]:
def reformat_dict(current_dict): # add in neutralization
                                # Get rid of H+ (cpd00067)
                                # compound IDs are five digits and rule IDs are four digits
                                # tables 6 and 8
    rxn_count = 0
    reformatted_dict = {}
    
    for i in range(0,132):
        
        rxn_count += 1
        key = 'R%i'%rxn_count
        value = list([])
        num_reactants = len(current_dict['Reaction %i Reactant ID'%i][0])
        num_products = len(current_dict['Reaction %i Products ID'%i][0])
        
        if num_reactants != 0 and num_products != 0: # ensure reactant and product list not empty
            
            reactant_dict = {} # initialize dict to store reactant info
            product_dict = {} # initialize dict to store product info
            
            for k in range(0,num_reactants):
                reactant_id = current_dict['Reaction %i Reactant ID'%i][0][0]
                reactant_smiles = current_dict['Reaction %i Reactant smiles'%i][0][0]
                reactant_dict.update({reactant_id:reactant_smiles})
                
            for j in range(0,num_products):
                product_id = current_dict['Reaction %i Products ID'%i][0][0]
                product_smiles = current_dict['Reaction %i Reactant smiles'%i][0]
                product_dict.update({product_id:product_smiles})
                
            value.append(reactant_dict)
            value.append(product_dict)
            reformatted_dict.update({key:value})
        
    return reformatted_dict

In [22]:
def map_rxn_to_rule(lhs_rhs):
    count = 0
    for i in range(0,2000):
        
        count += 1
        if count<10:
            rule = 'rule000%i'%count
        elif count >=10 and count<100:
            rule = 'rule00%i'%count
        elif count >=100 and count<1000:
            rule = 'rule0%i'%count
        else:
            rule = 'rule%i'%count
        
        try:
            mapping_object = mapper.map_pickaxe_rules(lhs_rhs[0], lhs_rhs[1],rule)
            if mapping_object[0][0] == 0 and mapping_object[1][0] == 0:
                break
        except ValueError: # exception for specific error type
            pass
        
    return rule

In [23]:
reformatted_dict = reformat_dict(table8_dict)
reformatted_dict

KeyError: 'Reaction 63 Reactant ID'

In [68]:
lhs_rhs = reformatted_dict['R28']
rule = map_rxn_to_rule(lhs_rhs)
rule

NameError: name 'reformatted_dict' is not defined

In [ ]:
for key in reformatted_dict.keys():
    lhs_rhs = reformatted_dict[key]
    map_rxn_to_rule(lhs_rhs)

In [ ]:
rxn_dict = {'R00465': 
            [{'cpd00006:0': 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)c1', 'cpd00139:0': 'O=C(O)CO'},
             {'cpd00005:0': 'NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1', 'cpd00040:0': 'O=CC(=O)O'},
             'rule0002']}

### dictionary structure
{'reaction number':[{'reactant_1_id':'reactant_1_smiles','reactant_2_id':'reactant_2_smiles'}.
                    {'product_1_id':'product_1_smiles','product_2_id':'product_2_smiles'},
                    'rule to try (iterable)']
}

### ALWAYS REMOVE cpd00067 H+

In [ ]:
table7_dict

In [ ]:
reactant_smiles = table7_dict['Reaction 0 Reactant smiles']

In [24]:
rxn_dict = {'R00465': 
            [{'cpd00006:0': 'NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)c1', 'cpd00139:0': 'O=C(O)CO'},
             {'cpd00005:0': 'NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)OC[C@H]3O[C@@H](n4cnc5c(N)ncnc54)[C@H](OP(=O)(O)O)[C@@H]3O)[C@@H](O)[C@H]2O)C=CC1', 'cpd00040:0': 'O=CC(=O)O'},
             'rule0002']}

In [31]:
def remove_hydrogen_ions(rxn_dict)
    for key in rxn_dict.keys():
        reactants = rxn_dict[key][0]
        products = rxn_dict[key][1]
        if 'cpd00067:0' in reactants.keys():
            del reactants['cpd00067']
        
        if 'cpd00067:0' in products.keys():
            del products['cpd00067']
    return rxn_dict